## Topographische und hydrographische Modelle (120.109)

### Protokoll 3

#### Aufgabe 2: Photoathymetrie (10 Punkte)

In der Aufgabe 2 sollen Sie ein DGM-W für den selben Ausschnitt mittels Photobathymetrie ableiten. Gegeben sind dazu vier Punktwolken *pho_\*.laz*, die mittels Dense Image Matching jeweils aus vier überlappenden UAV-Luftbildern (20-23, 21-24, 22-25, 23-26) abeleitet wurden. In einem Vorverarbeitungsschritt wurden aus den jeweiligen Gesampunktwolken die Bodenpunkte extrahiert. Für jeden der vier Bildblöcke liegt auch eine Textdatei mit den Projektionszentren der jeweiligen Bilder vor (*eor_\*.txt*). Für die Orientierungsdateien liegt auch eine Formatdefinitionsdatei vor (*photoBathyOriFmt.xml*). Alle genannten Dateien stehen wie bereits in Aufgabe 1 im Verzeichnis *~/shared/Daten/Protokoll_3* bereit.

Importieren Sie zunächst die Punktwolkendateien mit dem OPALS Modul [Import](https://opals.geo.tuwien.ac.at/html/stable/ModuleImport.html) und speichern Sie diese im aktuellen Verzeichnis unter dem selben Namen mit der (Standard)Endung *.odm*. Führen Sie danach die Refraktionskorrektur mit dem Modul [Snellius](https://opals.geo.tuwien.ac.at/html/stable/ModuleSnellius.html) durch. Dazu ist neben der Angabe des Refraktionsindex (*1.33*) und der Wasserspiegellage (*260.10*) für jede Punktwolke die korrespondierende Bildorientierungsdatei sowie die zugehörige Formatdefinition erforderlich. Speichern Sie die Ergebnisse für jede Punktwolke im aktuellen Verzeichnis in einem neuen ODM mit dem Suffix *_refCorr* (z.B. *pho_020_021_022_023_refCorr.odm*). Exportieren Sie anschließend alle refraktionskorrigierten ODMs in einen Gesamt-ODM und speichern Sie diesen im aktuellen Verzeichnis unter dem Dateinamen *photobathy_refCorr.odm*.

Interpolieren Sie schließlich ein DGM-W mit dem Modul [Grid](https://opals.geo.tuwien.ac.at/html/stable/ModuleSnellius.html). Verwenden Sie die selben Interpolationsparameter wie für das Laserbathymetrie DGM-W und speichern Sie das Rastermodell unter dem Dateinamen *photobathy_dgmw.tif*. Erzeugen Sie analog zu Aufgabe 1:
- eine DGM-W Schummerung: *photobathy_dgmw_shd.tif*
- eine DGM-W Höhenkodierung: *photobathy_dgmw_zco.tif*
- ein Wassertiefenmodell: *photobathy_wtiefe.tif*
- eine Höhenkodierung des Wassertiefenmodells: *photobathy_wtiefe_zco.tif*
- Höhenlinien des Wassertiefenmodells: *photobathy_wtiefe_iso.shp*

In [1]:
#
# Import Statements
#
import rasterio
import shapely
import fiona
import os

###################################################################
# Pfad zum Datenverzeichnis
###################################################################
datadir = os.path.join( os.getenv('HOME'), 'shared', 'Daten', 'Protokoll_3')

In [2]:
###################################################################
# Sicherstellung der Nicht-Existenz der Ergbnisdateien
###################################################################
!rm -f *.odm
!rm -f *.svg
!rm -f *.log
!rm -f photobathy_wtiefe*
!rm -f photobathy_dgmw*
!rm -f opalsLog.xml

In [3]:
# weitere Modulimporte
import opals
from opals import Import, Snellius, AddInfo, Grid, Shade, ZColor, Algebra, Isolines, Translate
from os.path import join

#Dateiennamen
dataset1 = 'pho_020_021_022_023'
dataset2 = 'pho_021_022_023_024'
dataset3 = 'pho_022_023_024_025'
dataset4 = 'pho_023_024_025_026'
dataset = 'photobathy'
lazF1 = join(datadir, dataset1 + '.laz')
lazF2 = join(datadir, dataset2 + '.laz')
lazF3 = join(datadir, dataset3 + '.laz')
lazF4 = join(datadir, dataset4 + '.laz')
odmF1 = dataset1 + '.odm'
odmF2 = dataset2 + '.odm'
odmF3 = dataset3 + '.odm'
odmF4 = dataset4 + '.odm'
odmF_corr1 = 'pho_020_021_022_023' + '_refCorr.odm'
odmF_corr2 = 'pho_021_022_023_024' + '_refCorr.odm'
odmF_corr3 = 'pho_022_023_024_025' + '_refCorr.odm'
odmF_corr4 = 'pho_023_024_025_026' + '_refCorr.odm'
odmF_corr = 'photobathy_refCorr.odm'

eorF1 = join(datadir, 'eor_020_021_022_023.txt')
eorF2 = join(datadir, 'eor_021_022_023_024.txt')
eorF3 = join(datadir, 'eor_022_023_024_025.txt')
eorF4 = join(datadir, 'eor_023_024_025_026.txt')
eofF = join(datadir, 'photoBathyOriFmt.xml')

dgmwF = dataset + '_dgmw.tif'
wdeF = dataset + '_wdepth.tif'
wlev = 260.10

# Datenimport
Import.Import(inFile = lazF1, outFile = odmF1).run()
Import.Import(inFile = lazF2, outFile = odmF2).run()
Import.Import(inFile = lazF3, outFile = odmF3).run()
Import.Import(inFile = lazF4, outFile = odmF4).run()

# Refraktionskorrektur
Snellius.Snellius(inFile = odmF1, outFile = odmF_corr1, refractiveIndex = 1.33, refModel = str(wlev), oriFile = eorF1, oriFormat = eofF).run()
Snellius.Snellius(inFile = odmF2, outFile = odmF_corr2, refractiveIndex = 1.33, refModel = str(wlev), oriFile = eorF2, oriFormat = eofF).run()
Snellius.Snellius(inFile = odmF3, outFile = odmF_corr3, refractiveIndex = 1.33, refModel = str(wlev), oriFile = eorF3, oriFormat = eofF).run()
Snellius.Snellius(inFile = odmF4, outFile = odmF_corr4, refractiveIndex = 1.33, refModel = str(wlev), oriFile = eorF4, oriFormat = eofF).run()

#Zusammenfügen der odms
Import.Import(inFile = [odmF_corr1, odmF_corr2, odmF_corr3, odmF_corr4], outFile = odmF_corr).run()

# DGM-W interpolieren
Grid.Grid(inFile = odmF_corr, outFile = dgmwF, interpolation = opals.Types.GridInterpolator.movingAverage, gridSize = 0.1, neighbours = 16, searchRadius = 2, selMode = opals.Types.SelectionMode.quadrant).run()


# Visualisierung des DGMW
Shade.Shade(inFile = dgmwF, outFile = 'photobathy_dgmw_shd.tif', sunPosition=[350,50]).run()
palF = 'standardPal.xml'
scaleP = '256 270'
ZColor.ZColor(inFile = dgmwF, outFile = 'photobathy_dgmw_zco.tif', interval = 0.5, palFile = palF, scalePal=scaleP).run()


# Wassertiefenmodell
Algebra.Algebra(inFile = dgmwF, formula = 'r[0] < 260.10 ? 260.10 - r[0] : invalid', outFile = wdeF).run()

#Visualisierung Wassertiefenmodell
palF = 'colorBrewer_SpectralPal.xml'
scaleP = '0 1.5'
ZColor.ZColor(inFile=wdeF, outFile='photobathy_wtiefe_zco.tif', palFile=palF, scalePal=scaleP, interval=0.1).run()
Isolines.Isolines(inFile=wdeF, outFile='photobathy_wtiefe_iso.shp', interval=0.1, minLength = 2, closeMin = 3).run()

IOStream.flush timed out


[Parameter(path=inFile, values=photobathy_wdepth.tif, type=Path, source=interFace),
 Parameter(path=outFile, values=photobathy_wtiefe_iso.shp, type=Path, source=interFace),
 Parameter(path=oFormat, values=shape, type=String, source=estimated),
 Parameter(path=band, values=0, type=String, source=internalDefault),
 Parameter(path=interval, values=0.10000000000000001, type=double, source=interFace),
 Parameter(path=levels, values=, type=Vector<double>, source=unknown),
 Parameter(path=zRange, values=, type=Array<double,2>, source=unknown),
 Parameter(path=thinOut, values=0.01, type=double, source=internalDefault),
 Parameter(path=closeMin, values=3, type=double, source=interFace),
 Parameter(path=minLength, values=2, type=double, source=interFace),
 Parameter(path=densification, values=1, type=bool, source=internalDefault)]

In [4]:
# Sichtbare Überprüfungen
assert os.path.exists('photobathy_refCorr.odm')
#
assert 'Import' in globals()
assert 'Snellius' in globals()
assert 'Grid' in globals()
assert 'Shade' in globals()
assert 'ZColor' in globals()
assert 'Algebra' in globals()
assert 'Isolines' in globals()

In [5]:
# Verborgene Überprüfungen 1 (DGM-W, photobathy_dgmw.tif)

In [6]:
# Verborgene Überprüfungen 2 (photobathy_dgmw_zco.tif)

In [7]:
# Verborgene Überprüfungen 3 (photobathy_dgmw_shd.tif)

In [8]:
# Verborgene Überprüfungen 4 (Wassertiefenmodell, photobathy_wtiefe.tif)

In [9]:
# Verborgene Überprüfungen 5 (photobathy_wtiefe_iso.shp, photobathy_wtiefe_zco.tif)

In [10]:
# Verborgene Überprüfungen 6  (Refraktionskorrektur Datei pho_020_021_022_023.odm, Min/Max)

In [11]:
# Verborgene Überprüfungen 7 (Refraktionskorrektur Datei pho_020_021_022_023.odm, Min/Max)

### Aufgabe 2b

Beschreiben Sie in ca 100-200 Worten kurz das Grundkonzept der Photobathymetrie (Mehrmedienphotogrammetrie) mit Fokus auf Refraktionskorrektur. Gehen Sie dabei auf die Strahlablenkung an der Luft-Wasser-Grenzschicht ein und diskutieren Sie insbesondere die generelle Richtung der Z-Korrektur in der Photobathymetrie.

Die Mehrmedienphotogrammetrie ist wie die Laserbathymetrie eine Methode, um Gewässertopografien zu erfassen. Sie ist jedoch anders als letztere eine passive Methode und nutzt die optische Strahlung, die auf und unter dem Wasser in Richtung Kamera reflektiert wird. 

Auch hier gibt es Probleme mit der Brechung von Licht, denn auch das reflektierte Licht unterliegt den Snellius’schen Brechungsgesetzen an der Wasser-Luft-Grenzschicht. Der Gewässerboden wird tendenziell zu hoch geschätzt, also die Gewässer scheinen ohne Refraktionskorrektur seichter zu sein als sie es tatsächlich sind. Ein weiteres Problem der Photobathymetrie ist, dass sie bei tieferen Gewässern nicht so gut eindringen kann wie das der Laser könnte. Die reine Sonneneinstrahlung ist dafür etwas zu schwach.